In [1]:
import os
import numpy as np

どう考えてもPandasを入れたほうがいい  
まあ作ったものは仕方ないので、読み出したあとdicをまとめる操作以降をPandasする

数値をtxtファイルに保存する関数を作る　trainer mainで呼ぶ  
１行目のdescription(default=#),2行目以降の　「属性：値」　は引数として自由に取る    
保存先ディレクトリとファイル接頭辞もとりあえず引数に取ることにして、訓練のmain.pyが参照する config.jsonにそのパスは入れておく形式で行く  
  
ファイルidをどうすればいいかわからないが、fileidを引数にとってそのファイルに書き込む形式にするとjoinも非joinも対応できるか  
やっぱ最初にid一覧を引き取って、そのぶんだけdescriptionを書く初期化を行うことにする  
非joinの場合のみ、1Storing()=1ファイル となる 

freeidについてはrunidは考慮しなくていい。freeid or babitaskidのことを指す    
-/ntm_vlgiitr/associative/rrnn/自由_runid1.txt  
-babi/rrnn/id1/task1.txt  
読み込む.jsonごとに、-/ntm_vlgiitr/associative までは書いておける  
モデルごとに分ける次の階層については、.pyに書きたいところだが繰り返し書くのも良くない  
 →utilとしてここに書くことにした。必要な引数を渡すとディレクトリ名を返す。比較対象モデルのデータはここに持つ感じで  
runidはtrain.pyの引数からscorestoringの引数へ
自由は今までfilename_idだったとこ　将来的に改善しそうな部分を書く。改善後は別モデルdirに分けられるけど  
commonは廃止。commonで表現すべきところはdir分けをする  
batch_size などはdescriptionに辞書形式で入れる  


In [2]:

import datetime
import json
class ScoreStoring(): #1ディレクトリ。引数はconfig only これはjointのときだけ使うことにする
    #data_config["savescore_dir"],data_config["filename_common"]
    def __init__(self, data_config,fileid_str_list,dir_modelname,runid,file_description,babi=False,dir_descrption=None):
        file_description["datetime"]=str(datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))))
        print("filename_ids:",fileid_str_list,"runid:",runid)
        #fileid_str_list : ["1","2","3"... ] など 日付にするのも良い〜
        self.filepath_dic ={}
        if (not babi) and len(fileid_str_list)!=1:raise ValueError("fileid list size must be 1 ohter than babi")
        if babi:raise ValueError("not implemented")
        for freeid in fileid_str_list:
            self.filepath_dic[freeid] =os.path.join(data_config["savescore_dir"],dir_modelname,freeid+"_runid"+str(runid)+".txt")
            print("score_path:",self.filepath_dic[freeid] )
            #fileがすでにあるならその中身を初期化し、１行目にdescriptionを書く
            with open( self.filepath_dic[freeid], "w" , encoding= "utf-8" ) as f:
                #f.write(file_description+rowAttr+":"+rowVal+"\n")
                f.write(json.dumps(file_description)+"\n")
        
    def store(self,fileid_str,attr,val):
        filepath=self.filepath_dic[fileid_str]
        with open( filepath, "a" , encoding= "utf-8" ) as f:
            f.write(str(attr)+":"+str(val)+"\n")

        ##writelines

#xとyが１；１以外でも、ｙが複数合っても対応できるようにしたい

In [3]:

class ScoreStoringLoader(): 
    #data_config["savescore_dir"],data_config["filename_common"]
    def __init__(self,savescore_dir,modeldir_list,fileid,runid_list,desc_lines=1,sep=":"):
        self.desc_lines =desc_lines
        self.sep=sep
        print("run_ids:",runid_list)
        self.runid_list_str =["runid"+str(rid) for rid in runid_list]
        if type(fileid)!=type(" "):raise ValueError("fileid must be str. 1 Loader makes 1 image")
        self.data_dic={} #{"rrnn":
                            #{"batch_size":1600,
                            # "runid1":{"epoch":[0,10,20...]  , "accuracy":[0.7,0.8,0.9 ...]  , "z"...}  
                            #}, "sortrrnn":...}

        for modeldir in modeldir_list:
            self.data_dic[modeldir] ={}
            for idx,rid in enumerate( self.runid_list_str):
                data_path=os.path.join(savescore_dir,modeldir,fileid+"_"+rid+".txt")
                with open(data_path, "r" , encoding= "utf-8" ) as f:
                    filecontent=f.read().splitlines()
                    file_description=json.loads(filecontent[0])
                    lines=filecontent[desc_lines:]

                    if idx==0:self.data_dic[modeldir]["batch_size"]=int(file_description["batch_size"])
                    if self.data_dic[modeldir]["batch_size"]!=int(file_description["batch_size"]):raise ValueError("inconsistent batch_size over runid")

                    listed_data = [line.split(self.sep) for line in lines]
                    listed_data = np.array([[float(data) for data in line] for line in listed_data])
                    self.data_dic[modeldir][rid] ={file_description["rowAttr"]:listed_data[:,0],file_description["rowVal"]:listed_data[:,1]}

    def get_data_dic(self): 
        return self.data_dic

In [4]:
def get_dirname(infer_type,sort_flag):
    dir_modelname=["ntm","rrnn","sortrrnn"]
    if infer_type==1:
        if sort_flag:
            return dir_modelname[2]
        else:
            return dir_modelname[1]
    else:
        return dir_modelname[0]



SyntaxError: invalid syntax (3131940118.py, line 1)